In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from tqdm.notebook import tqdm

<img src = "LeNet5_architecture.png">

We'll change some parts like subsampling to MaxPool and Euclidean distance to Cross Entropy Loss. Also we used ReLU instead of a sigmoid activation. These changes are what I call "modern" approaches because through the year these have been shown to outperform the other methods.

In [2]:
mnist = datasets.MNIST(root = "../data", download = True)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
bs = 100

In [4]:
class MNISTTrainingDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, mean, std):
        super().__init__()
        train = data.reshape((-1,1,28,28))
        flat = train.flatten().to(torch.float32)

        self.train = ((train - mean)/ std).to(device)
        self.label = targets.to(torch.long).to(device)
        
    def __getitem__(self, i):
        return self.train[i], self.label[i]
    
    def __len__(self):
        return self.train.shape[0]

In [5]:
def train_test_datasets():
    #we have to use the training mean on the test set
    train = mnist.data[:50000].reshape((-1,1,28,28))
    flat = train.flatten().to(torch.float32)
    mean = flat.mean()
    std = flat.std()
    
    return MNISTTrainingDataset(mnist.data[:50000], mnist.targets[:50000], mean,std),\
        MNISTTrainingDataset(mnist.data[50000:], mnist.targets[50000:], mean, std)   

In [6]:
mnist_train, mnist_test = train_test_datasets()

In [7]:
train_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size = bs, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size = bs, shuffle = False)

In [8]:
class LeNetModern(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding = 2)
        self.subsampling1 = nn.MaxPool2d(2 , 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.subsampling2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(16,120,5)
        self.f1 = nn.Linear(120, 84)
        self.f2 = nn.Linear(84,10)
        
    def forward(self, x):
        C1 = self.conv1(x)
        S2 = F.relu(self.subsampling1(C1))
        C3 = self.conv2(S2)
        S4 = F.relu(self.subsampling2(C3))
        C5 = F.relu(self.conv3(S4).squeeze())
        F6 = F.relu(self.f1(C5))
        return self.f2(F6)
        

In [9]:
lenet_modern = LeNetModern().cuda()

In [10]:
loss = nn.CrossEntropyLoss()
optim = torch.optim.SGD(lenet_modern.parameters(), lr=0.03)

In [11]:
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optim, lambda e: 0.95)

In [12]:
def get_accuracy():
    accumulated_accuracy = []
    for batch, truth in test_dataloader:
        weighted_rate = (lenet_modern(batch).argmax(dim = 1) == truth).sum().cpu().numpy()
        accumulated_accuracy.append(weighted_rate)
    return sum(accumulated_accuracy)/bs/len(test_dataloader)

In [13]:
accuracy = []

In [16]:
#training
epochs = 30

for epoch in tqdm(range(epochs)):
    temp = get_accuracy()
    accuracy.append(temp)
    print(temp)
    for batch, preds in train_dataloader:
        y_pred = lenet_modern(batch)
        
        l = loss(y_pred, preds)
        
        l.backward()
        
        optim.step()
        
        optim.zero_grad()


0.9991599999999999
0.9994
0.9995
0.9986
0.9994400000000001
0.99836
0.9990800000000001
0.99972
0.9997999999999999
0.99982
0.9998400000000001
0.99982
0.99986
0.99994
0.99998
0.99994
0.9999600000000001
0.9999
0.99988
0.99994
0.99998
0.99964
0.9999600000000001
0.99998
0.9997999999999999
1.0
0.99988
1.0
0.99994
1.0



In [15]:
accuracy

[0.10804000000000001,
 0.9332999999999999,
 0.9632200000000001,
 0.97072,
 0.98066,
 0.98262,
 0.9855,
 0.988,
 0.9885,
 0.9916799999999999,
 0.99222,
 0.99142,
 0.992,
 0.9940399999999999,
 0.99482,
 0.9946200000000001,
 0.9947,
 0.9950800000000001,
 0.99414,
 0.99598,
 0.99454,
 0.9969199999999999,
 0.9979199999999999,
 0.99774,
 0.996,
 0.9986,
 0.99822,
 0.998,
 0.99862,
 0.99878]

In [ ]:
mnist["test"]

In [ ]:
foo.shape

In [16]:
foo=next(iter(dataloader))[0].type(torch.float32)

NameError: name 'dataloader' is not defined

In [70]:
kernel = torch.tensor([[[0,0],[0,0]] for _ in range(5)]).unsqueeze(1).type(torch.float32)

In [71]:
kernel.shape

torch.Size([5, 1, 2, 2])

In [72]:
F.conv2d(foo, kernel)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          

In [46]:
foo


tensor([[[[-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          ...,
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241]]],


        [[[-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          ...,
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241]]],


        [[[-0.4241, -0.4241, -0.4241,  ..., -0.4241, -0.4241, -0.4241],
          [-0.4241, -0.424

In [17]:
for parameter in lenet_modern.parameters():
    print(parameter.data.shape)

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 16, 5, 5])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])
